In [13]:
import tablescraper as scrape
import requests
import re

In [4]:
bdays = scrape.scrape_elems(
    ['https://en.wikipedia.org/wiki/Robert_Downey_Jr.', 'https://en.wikipedia.org/wiki/Paul_Bettany'],
    '//*[@class="bday"]/text()')

bdays

['1965-04-04', '1971-05-27']

## Get wikipedia page by actor_name, then get birthday

In [28]:
def wikipedia_url(actor_names):

    actor_names = scrape.to_sequence(actor_names)
    final_urls = []
    
    for actor_name in actor_names:

        url = 'https://en.wikipedia.org/w/index.php?search={name}&title=Special%3ASearch&go=Go'.format(
            name=re.sub(r"[^A-Za-z+]+", '', actor_name.lower().replace(' ','+')))

        final_urls.append(url)

    return final_urls

actor_names = ['Robert Downey, Jr.', 'Ben Foster', 'Mireille Enos']
wiki_urls = wikipedia_url(actor_names)
birthdays = scrape.scrape_elems(wiki_urls, '//*[@class="bday"]/text()')

for a,b,c in zip(actor_names,birthdays, wiki_urls):
    print a,b,c

Robert Downey, Jr. 1965-04-04 https://en.wikipedia.org/w/index.php?search=robert+downey+jr&title=Special%3ASearch&go=Go
Ben Foster 1980-10-29 https://en.wikipedia.org/w/index.php?search=ben+foster&title=Special%3ASearch&go=Go
Mireille Enos 1975-09-22 https://en.wikipedia.org/w/index.php?search=mireille+enos&title=Special%3ASearch&go=Go


In [8]:
# This doesn't work without a browser
# 
# 
# google_urls = ['http://www.google.com/search?q={name}+birthday'.format(name=name) 
#         for name in ['paulbettany','jeffbridges']]

# print google_urls

# google_bdays = scrape.scrape_elems(google_urls, '//*[@class="_XWk"]', 0)

# google_bdays

['http://www.google.com/search?q=paulbettany+birthday', 'http://www.google.com/search?q=jeffbridges+birthday']


IndexError: list index out of range

# Gender? 

You could calculate the "her"+"she" to "his"+"he" ratio in the Wikipedia page text...

# Name linking?
